<a href="https://colab.research.google.com/github/Huwalli/PLta/blob/main/%E7%B6%B2%E8%B7%AF%E8%B3%87%E6%96%99%E8%87%AA%E5%8B%95%E7%88%AC%E5%8F%96%E8%88%87%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ 1. 從網路上抓取新聞（Web Scraping）

以 ptt 為例，使用 requests + BeautifulSoup 抓取新聞標題與內容。

✅ 2. 分析新聞內容（NLP 分析）

用 SnowNLP（中文）進行情緒分析。

✅ 3. 視覺化分析結果

用 pandas 做表格處理，plotly 視覺化結果。

In [1]:
!pip install snownlp plotly beautifulsoup4 # 安裝中文 NLP、互動視覺化與 HTML 解析套件

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=fcdabc54d9371dec19a49e390b55cd8e7600991d2be7d1d97d12e3b5c942f86c
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [2]:
# 匯入 requests 模組，用來對網頁伺服器發送請求 (Request) 並取得回應 (Response)
import requests

# 從 bs4 套件中匯入 BeautifulSoup，用來解析 HTML 文件，方便從網頁中找出特定資料
from bs4 import BeautifulSoup

# 匯入 SnowNLP 套件，用來進行中文的自然語言處理（像是情感分析、斷詞）
from snownlp import SnowNLP

# 匯入 pandas，用來整理資料成表格（DataFrame），方便後續分析與儲存
import pandas as pd

# 匯入 plotly.express，這是畫互動圖表的套件，可以用來視覺化分析結果
import plotly.express as px

# 設定瀏覽器標頭（headers），模擬成一般使用者開啟網頁，避免被網站封鎖
HEADERS = {
    'User-Agent': 'Mozilla/5.0'  # 模擬成常見的瀏覽器（像是 Chrome）
}

# 設定 cookies，這是針對某些版面（如 PTT Gossiping）會要求確認年齡，這裡模擬已經按下「我已滿18歲」
COOKIES = {'over18': '1'}


In [3]:
# 定義函式 get_ptt_articles，預設抓取 PTT Stock 板的前 2 頁文章
def get_ptt_articles(board='Stock', max_pages=2):
    # 設定第一頁的網址
    base_url = f'https://www.ptt.cc/bbs/{board}/index.html'

    # 用來儲存所有文章資料的清單
    articles = []

    # 重複爬 max_pages 頁
    for _ in range(max_pages):
        # 發送 GET 請求取得該頁 HTML 原始碼，附上 headers 和 cookies 模擬使用者
        res = requests.get(base_url, headers=HEADERS, cookies=COOKIES)
        # 用 BeautifulSoup 解析 HTML
        soup = BeautifulSoup(res.text, 'html.parser')
        # 找出所有文章的區塊（div.r-ent 是每篇文章的外層容器）
        entries = soup.select('div.r-ent')

        # 逐篇處理文章
        for entry in entries:
            # 找出文章標題的 <a> 標籤
            title_tag = entry.select_one('div.title > a')
            if title_tag:
                # 取得文章標題文字，去除前後空白
                title = title_tag.text.strip()
                # 製作完整的文章網址
                link = 'https://www.ptt.cc' + title_tag['href']

                # 這整段都在作文章情緒的分析(如果不想做情緒分析也可拿掉)
                try:
                    # 再次發送請求進入文章頁面
                    article_res = requests.get(link, headers=HEADERS, cookies=COOKIES)
                    article_soup = BeautifulSoup(article_res.text, 'html.parser')
                    # 抓取主內容區（#main-content）並切除簽名檔（用 -- 分隔）
                    content = article_soup.select_one('#main-content').text.split('--')[0]
                    # 去除以「※」開頭的系統資訊
                    content = '\n'.join(line for line in content.split('\n') if not line.startswith('※'))
                    # 用 SnowNLP 計算情緒分數（0 為負面，1 為正面）
                    sentiment = SnowNLP(content).sentiments
                except Exception as e:
                    # 若發生錯誤（例如請求失敗），則將內容設為空，情緒分數為 None
                    content = ""
                    sentiment = None

                # 將資料加入結果清單
                articles.append({
                    '標題': title,
                    '連結': link,
                    '情緒分數': sentiment
                })

        # 嘗試找出「上一頁」的連結（第二個按鈕通常是上一頁）
        paging = soup.select_one('div.btn-group-paging > a.btn.wide:nth-child(2)')
        if paging:
            # 更新 base_url 為上一頁的網址，準備下一輪爬取
            base_url = 'https://www.ptt.cc' + paging['href']
        else:
            # 如果找不到，跳出迴圈
            break

    # 最後將所有資料轉換為 pandas 資料表並回傳
    return pd.DataFrame(articles)


In [4]:
# 抓 PTT Stock 看板 2 頁
df = get_ptt_articles('Stock', max_pages=2)
df = df.dropna(subset=['情緒分數'])

In [5]:
df

,標題,連結,情緒分數
0,[新聞] 這檔連飆3根漲停 6千張排隊等買！,https://www.ptt.cc/bbs/Stock/M.1749787827.A.B6...,1.332268e-15
1,Re: [新聞] 伊朗預告「嚴厲報復」！美駐以色列使館,https://www.ptt.cc/bbs/Stock/M.1749788060.A.22...,3.625562e-01
2,[新聞] 凱基金睽違13年再配股！股利0.95元創次,https://www.ptt.cc/bbs/Stock/M.1749788093.A.8D...,0.000000e+00
3,[新聞] 以色列襲擊伊朗 川普承認事先知情、重申,https://www.ptt.cc/bbs/Stock/M.1749790429.A.75...,2.672384e-07
4,[標的] TSLL.US Robotaxi多,https://www.ptt.cc/bbs/Stock/M.1749791391.A.45...,5.607279e-04
5,[新聞] 麥味登兄弟公司、北台灣最大家禽電宰場,https://www.ptt.cc/bbs/Stock/M.1749794167.A.A9...,4.440892e-16
6,[請益] 一朗安安靜靜的，怎摸被炸？,https://www.ptt.cc/bbs/Stock/M.1749794369.A.5E...,3.010478e-04
7,[閒聊] 2025/06/13 盤後閒聊,https://www.ptt.cc/bbs/Stock/M.1749794402.A.C8...,0.000000e+00
8,[新聞] 關稅豁免期延定了？謝金河解讀「川普魔術,https://www.ptt.cc/bbs/Stock/M.1749796258.A.38...,1.000000e+00
9,Re: [請益] 為何股息要計入所得課稅？,https://www.ptt.cc/bbs/Stock/M.1749796360.A.70...,9.999968e-01


In [6]:
# 畫圖
fig = px.bar(
    df.sort_values('情緒分數'),
    x='情緒分數',
    y='標題',
    orientation='h',
    title='PTT 文章情緒分析',
    height=600
)
fig.update_layout(yaxis=dict(automargin=True))
fig.show()